In [1]:
from rich import print
import os
from openai import OpenAI
from datasets import Dataset, DatasetDict, load_dataset
import json
from dotenv import load_dotenv
import time
from tenacity import retry, stop_after_attempt, wait_exponential

# Load environment variables from .venv file
load_dotenv() 

f:\UNIBO\4 Sem\Docker\docker_commands\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [35]:
topic = "Docker"
n_subtopics = 5
n_instructions = 100


client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.environ["NVIDIA_API_KEY"]
)

# 1. Subtopics Generation

In [36]:
TOPIC_GENERATION_PROMPT_TEMPLATE = """\
I want to create a dataset of Docker commands. Based on this, generate a list of {n_subtopics} subtopics to cover what needs to be covered when working with Docker.

The topic is: {topic}

The list must be without numbers, and without any description of the subtopics. The subtopics should be separated by a comma. There must be no other text than the list.
"""

def generate_subtopics(client, topic, n_subtopics):
    prompt = TOPIC_GENERATION_PROMPT_TEMPLATE.format(topic=topic, n_subtopics=n_subtopics)
    response = client.chat.completions.create(
        model="meta/llama-3.1-405b-instruct",
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    return response

response = generate_subtopics(client, topic=topic, n_subtopics=n_subtopics)
print(response.choices[0].message.content)

Images, Containers, Volumes, Networking, Security

# 2. Questions Generation

In [37]:
QUESTION_PROMPT_TEMPLATE = """\
To create a dataset of Docker commands. Given a topic in Docker, generate {n_instructions} instructions that could be asked about that topic. Your response should be in a list format.

The topic is: {sub_topic}

The list must be without numbers. The questions/instructions should be separated by a newline character. There must be no other text than the list.
"""
subtopic_list = response.choices[0].message.content.split(",")
def generate_questions(client, sub_topic, n_instructions):
    prompt = QUESTION_PROMPT_TEMPLATE.format(sub_topic=sub_topic, n_instructions=n_instructions)
    response = client.chat.completions.create(
        model="meta/llama-3.1-405b-instruct",
        messages=[
            {"role": "user",
             "content": prompt}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
    )
    return response.choices[0].message.content

def instruction_generator(client, subtopic_list, n_instructions):
    instruction_list = [generate_questions(client, subtopic, n_instructions) for subtopic in subtopic_list]
    return instruction_list

instruction_list = instruction_generator(client, subtopic_list, n_instructions)

instruction_list_formatted = []
for instruction_set in instruction_list:
    instruction_list_formatted.extend([question.strip() for question in instruction_set.split("\n") if question])
len(instruction_list_formatted)

482

# 3. Response Generation

In [38]:
RESPONSE_PROMPT_TEMPLATE = """\
Given a question/instruction about Docker, generate response that could be given.

The user prompt is: {instruction}

"""
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=4, max=60))
def generate_response(client, instruction):
    prompt = RESPONSE_PROMPT_TEMPLATE.format(instruction=instruction)
    try:
        response = client.chat.completions.create(
            model="meta/llama-3.1-405b-instruct",
            messages=[
                {"role": "user",
                "content": prompt}
            ],
            temperature=0.2,
            top_p=0.7,
            max_tokens=1024,
        )
        print(response.choices[0].message.content)
        return response.choices[0].message.content
    except Exception as e:
        if "429" in str(e):
            print(f"Rate limit hit, waiting before retry...")
            time.sleep(5)  # Add extra delay for rate limits
        raise e

def response_generator(client, instruction_list):
    tasks = [generate_response(client, instruction) for instruction in instruction_list]
    response_list = tasks
    return response_list

instruction_response_list = response_generator(client, instruction_list_formatted)
instruction_response_pair_list = []
for instruction, response in zip(instruction_list_formatted, instruction_response_list):
    try:
        instruction_response_pair_list.append(
            {
                "instruction": instruction,
                "response": response
            }
        )
    except Exception as e:
        if "429" in str(e):
            print(f"Rate limit hit, waiting before retry...")
            time.sleep(5)  # Add extra delay for rate limits
        raise e

To create a new Docker image from a Dockerfile in the current directory, you can use the Docker `build` command. 
The basic syntax for this command is:

```bash
docker build -t your-image-name .
```

Here's a breakdown of the components of this command:

- `docker build`: This is the command to build a Docker image from a Dockerfile.
- `-t your-image-name`: The `-t` flag is used to specify the name (tag) of your Docker image. Replace 
`your-image-name` with the desired name for your image. You can also include a version number by appending 
`:version` to the name, e.g., `your-image-name:1.0`.
- `.`: The dot at the end of the command tells Docker to look for the Dockerfile in the current directory. Make 
sure you are in the directory containing your Dockerfile when you run this command.

### Example:

If you want to create a Docker image named `myapp` with the version `1.0`, you would use the following command:

```bash
docker build -t myapp:1.0 .
```

After running this command, Docker will build the image based on the instructions in your Dockerfile. Once the 
build process is complete, you can verify that the image has been created by listing all your Docker images with 
the command:

```bash
docker images
```

This will display a list of your Docker images, including the one you just created. You can then run a container 
from this image using the `docker run` command, specifying the image name and version:

```bash
docker run myapp:1.0
```

Replace `myapp:1.0` with the name and version of your Docker image.

To build a Docker image with a specific tag, you can use the `docker build` command followed by the `-t` flag, 
which specifies the tag you want to assign to your image. The basic syntax for this command is:

```bash
docker build -t your-tag-name .
```

Here's a breakdown of the command:

- `docker build`: This is the command to build a Docker image.
- `-t`: This flag is used to specify the tag for the image. You can think of the tag as the name of your image.
- `your-tag-name`: Replace this with the specific tag you want to use for your image. It's common to use a format 
like `username/image-name` for the tag, especially if you plan to push the image to Docker Hub.
- `.`: The dot at the end of the command tells Docker to look for the Dockerfile in the current directory. If your 
Dockerfile is located in a different directory, you would replace the dot with the path to that directory.

### Example

Let's say you want to build a Docker image for a simple web application, and you want to tag it as 
`myuser/mywebapp`. Assuming your Dockerfile is in the current directory, you would run:

```bash
docker build -t myuser/mywebapp .
```

After running this command, Docker will execute the instructions in your Dockerfile and create an image with the 
specified tag. You can then use this tag to run containers from the image, push the image to a registry, or use it 
as a base for other images.

### Additional Tips

- **Specify a Dockerfile Location**: If your Dockerfile is not in the current directory, you can specify the path 
to it at the end of the command instead of the dot. For example, if your Dockerfile is in a directory named 
`myproject`, you would use `docker build -t myuser/mywebapp myproject`.
- **Use Multiple Tags**: You can apply multiple tags to an image by repeating the `-t` flag. This can be useful if 
you want to tag an image with different versions or names. For example, `docker build -t myuser/mywebapp:latest -t 
myuser/mywebapp:v1.0 .`
- **Pushing to Docker Hub**: If you plan to push your image to Docker Hub, make sure the tag starts with your 
Docker Hub username, followed by the image name, and optionally, a version tag. After building, you can push the 
image using `docker push your-tag-name`.

To list all available Docker images on your system, you can use the `docker images` command. This command will 
display a list of all Docker images that are currently stored on your system, including their repository names, 
tags, image IDs, creation dates, and sizes.

Here is an example of how to use the `docker images` command:

```
docker images
```

When you run this command, Docker will display a list of all available images in a format similar to the following:

```
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
ubuntu              latest              4e2eef94cd6b        4 weeks ago         72.9MB
nginx               latest              231d40e811cd        4 weeks ago         126MB
postgres            latest              7f6a5f0f1b7a        4 weeks ago         313MB
```

In this example, the output shows three Docker images: `ubuntu`, `nginx`, and `postgres`, each with their 
respective tags, image IDs, creation dates, and sizes.

If you want to list all images, including intermediate images that are not tagged, you can use the `-a` flag with 
the `docker images` command:

```
docker images -a
```

This will display a more comprehensive list of all Docker images on your system, including intermediate images that
are not explicitly tagged.

To delete a Docker image by its ID, you can use the Docker `rmi` command followed by the ID of the image you want 
to delete. Here's how you can do it:

1. **First, find the ID of the image you want to delete** by listing all Docker images:

```bash
docker images
```

This command will list all available Docker images on your system, including their IDs.

2. **Delete the Docker image** using its ID:

```bash
docker rmi <image-id>
```

Replace `<image-id>` with the actual ID of the image you want to delete. If the image has multiple tags and you 
want to delete all tags associated with the image, you can use the `-f` option to force delete:

```bash
docker rmi -f <image-id>
```

**Note:** Be cautious when deleting Docker images, as this action is irreversible. Make sure you no longer need the
image or have a way to recreate it before deleting it.

**Additional Tip:** If you're trying to free up disk space, consider also removing unused Docker volumes and 
networks, as they can also consume significant space. You can remove unused data with:

```bash
docker system prune
```

This command removes unused data (i.e., volumes, networks, and images). Use it with caution, as it also removes 
data that might be necessary for some applications.

To remove all unused Docker images, you can use the Docker system prune command. This command will remove all 
unused data (i.e., volumes, networks, and images). If you specifically want to target images, you can use the 
`docker image prune` command. Here's how you can do it:

1. **Remove all unused data (including images):**
   ```
   docker system prune
   ```
   This command will prompt you for confirmation before deleting the data. If you want to bypass the confirmation 
prompt, you can use the `-f` or `--force` flag:
   ```
   docker system prune -f
   ```

2. **Remove only unused images:**
   ```
   docker image prune
   ```
   Similar to the system prune command, you can use the `-f` or `--force` flag to bypass the confirmation prompt:
   ```
   docker image prune -f
   ```

3. **Remove all unused images, not just dangling ones:**
   By default, `docker image prune` only removes dangling images (images that are not tagged and not referenced by 
any container). If you want to remove all unused images (including those that are tagged but not used by any 
container), you can use the `-a` or `--all` flag:
   ```
   docker image prune -a
   ```
   Again, you can use the `-f` flag to force the removal without a confirmation prompt:
   ```
   docker image prune -af
   ```

**Note:** Be cautious when using these commands, especially with the `--force` flag, as they will permanently 
delete data. Make sure you have backed up any important data before proceeding.

...

To use Docker's secure pid features, you can leverage the `--pid` flag when running a Docker container. Docker 
provides a few ways to manage process IDs (PIDs) to enhance security and isolation. Here's how you can utilize 
these features:

### 1. Using `--pid=host` for Shared PID Namespace

When you use `--pid=host`, the container shares the same PID namespace as the host. This means that the container 
can see and manipulate processes on the host system, which can be useful for debugging but should be used with 
caution due to the security implications.

```bash
docker run -it --pid=host my_image
```

### 2. Using `--pid` for PID Isolation

For enhanced security, Docker allows you to isolate a container's PID namespace. By default, Docker uses a private 
PID namespace for containers, but you can explicitly set it for clarity or when working with specific requirements.

```bash
docker run -it --pid my_container_pid_name my_image
```

### 3. Limiting PIDs with `--pids-limit`

To prevent a fork bomb or to limit the number of processes a container can spawn, you can use the `--pids-limit` 
flag. This enhances security by preventing a container from consuming all available PIDs on the host.

```bash
docker run -it --pids-limit 100 my_image
```

### 4. Using Docker Compose for PID Management

If you're using Docker Compose, you can configure PID settings in your `docker-compose.yml` file. For example, to 
set a PID limit:

```yaml
version: '3'
services:
  my_service:
    image: my_image
    pids_limit: 100
```

### Best Practices

- **Isolation**: Use Docker's PID isolation features to ensure that containers cannot see or affect processes on 
the host or in other containers, enhancing security.
- **Limit PIDs**: Always limit the number of PIDs available to a container to prevent abuse and potential 
denial-of-service attacks.
- **Monitoring**: Regularly monitor your containers' PID usage to detect any anomalies or potential security 
issues.

By following these guidelines and using Docker's secure PID features, you can enhance the security and reliability 
of your containerized applications.

**Configuring PID Mode of a Docker Container Securely**

By default, Docker containers run in their own PID namespace, which isolates them from the host system and other 
containers. However, in certain scenarios, you may need to configure the PID mode of a container to share the 
host's PID namespace or use a custom PID namespace. Here's how to do it securely:

**Option 1: Sharing Host PID Namespace (PID=host)**

To share the host's PID namespace, you can use the `--pid=host` flag when running a container:
```bash
docker run -d --pid=host my_image
```
This allows the container to see and interact with processes running on the host system. However, this also 
increases the attack surface, as a compromised container could potentially affect the host system.

**Option 2: Using a Custom PID Namespace (PID=container:<name>)**

To use a custom PID namespace, you can specify the `--pid` flag with the `container:<name>` option:
```bash
docker run -d --pid=container:my_container my_image
```
This creates a new PID namespace for the container, isolated from the host system and other containers.

**Security Considerations**

When configuring the PID mode of a container, keep the following security considerations in mind:

* **Least Privilege Principle**: Only grant the necessary privileges to the container. If the container doesn't 
need to interact with the host system, use the default PID namespace or a custom PID namespace.
* **Isolation**: Use a custom PID namespace to isolate the container from the host system and other containers.
* **Monitoring**: Regularly monitor the container's activity and logs to detect any potential security issues.

**Example Use Case**

Suppose you have a containerized web server that needs to interact with a process running on the host system. You 
can use the `--pid=host` flag to share the host's PID namespace:
```bash
docker run -d --pid=host -p 80:80 my_web_server_image
```
However, to minimize the attack surface, consider using a custom PID namespace and implementing additional security
measures, such as network policies and monitoring.

**Additional Resources**

For more information on Docker container configuration and security, refer to the official Docker documentation:

* [Docker Container Configuration](https://docs.docker.com/engine/reference/commandline/run/)
* [Docker Security](https://docs.docker.com/engine/security/)

To use Docker's secure IPC (Inter-Process Communication) features, you can leverage the following approaches:

1. **Docker Networking**: Docker provides a built-in networking system that allows containers to communicate with 
each other securely. You can create a Docker network and attach containers to it, enabling them to exchange data 
without exposing ports to the host machine.

Example:
```bash
docker network create my-network
docker run -it --net=my-network my-container
```
2. **Docker Compose**: Docker Compose is a tool for defining and running multi-container Docker applications. It 
provides a secure way to enable IPC between containers by automatically creating a network for the services defined
in the `docker-compose.yml` file.

Example:
```yaml
version: '3'
services:
  service1:
    build: .
    ports:
      - "8080:8080"
  service2:
    build: .
    depends_on:
      - service1
```
3. **Docker Volumes**: Docker Volumes provide a secure way to share data between containers. You can create a 
volume and mount it to multiple containers, allowing them to access and share data without exposing it to the host 
machine.

Example:
```bash
docker volume create my-volume
docker run -it -v my-volume:/shared-data my-container
```
4. **Docker Socket**: Docker provides a socket-based IPC mechanism that allows containers to communicate with each 
other. You can use the `docker exec` command to execute a command in a running container and use the `--socket` 
flag to specify the socket file.

Example:
```bash
docker exec -it my-container --socket /var/run/docker.sock my-command
```
5. **Secure Docker IPC using TLS**: Docker provides support for TLS encryption for secure IPC between containers. 
You can generate certificates and configure Docker to use them for encrypting IPC traffic.

Example:
```bash
docker run -it --tlsverify --tlscacert=ca.pem --tlscert=cert.pem --tlskey=key.pem my-container
```
By using these features, you can ensure secure IPC between Docker containers and protect sensitive data from 
unauthorized access.

**Best Practices:**

* Always use secure Docker networking and volumes to enable IPC between containers.
* Use Docker Compose to define and run multi-container applications with secure IPC.
* Use TLS encryption for secure IPC between containers.
* Limit the use of Docker socket-based IPC to specific use cases where it is necessary.
* Regularly review and update your Docker configuration to ensure it remains secure.

Configuring the IPC (Inter-Process Communication) mode of a Docker container securely is crucial to prevent 
unauthorized access and ensure the isolation of processes within the container. Docker provides several IPC modes 
that can be configured depending on your specific requirements. Here's how you can configure the IPC mode securely:

### 1. **Default (Private) Mode**

By default, Docker containers are started with the `private` IPC mode, where the container has its own private IPC 
namespace. This is the most secure mode as it isolates the container's processes from those on the host and other 
containers.

```bash
docker run -it --ipc=private myimage
```

However, since `private` is the default, you usually don't need to specify it explicitly.

### 2. **Host Mode**

Using `host` mode allows the container to use the host's IPC namespace. This is less secure than the default mode 
because it allows processes in the container to communicate with those on the host. Use this mode with caution.

```bash
docker run -it --ipc=host myimage
```

### 3. **Shareable Mode**

Docker also allows containers to share the IPC namespace with other containers. This can be useful for specific 
scenarios where containers need to communicate with each other, but it introduces security risks by allowing 
inter-container communication. To use this mode securely, ensure that only trusted containers share the IPC 
namespace.

First, create a container with a shareable IPC namespace:

```bash
docker run -it --ipc=shareable --name=shared-ipc-container myimage
```

Then, you can share this IPC namespace with another container:

```bash
docker run -it --ipc=container:shared-ipc-container anotherimage
```

### Best Practices for Secure IPC Configuration

- **Use the Default Private Mode**: Unless you have a specific requirement, stick with the default `private` IPC 
mode for maximum isolation and security.
- **Limit Use of Host Mode**: Only use `host` mode when absolutely necessary, as it breaks the isolation model and 
can expose the host to potential risks.
- **Be Cautious with Shareable Mode**: When using `shareable` mode, ensure that you only share the IPC namespace 
among trusted containers to prevent unauthorized access.
- **Monitor and Audit**: Regularly monitor and audit your container configurations to ensure that IPC modes are 
configured as intended and that no unauthorized changes have been made.

By following these guidelines, you can configure the IPC mode of your Docker containers securely, maintaining the 
isolation and security of your containerized applications.

It looks like you're interested in learning how to use Docker. Docker is a powerful tool for containerizing 
applications, making it easier to develop, deploy, and manage applications in a variety of environments. Here's a 
basic guide to get you started:

### 1. Installing Docker

First, you need to install Docker on your system. Docker provides a comprehensive guide for installation on various
operating systems including Windows, macOS, and Linux. You can find these guides on the official Docker website.

- **For Windows and macOS**: Docker Desktop is the recommended application for managing Docker on these platforms. 
It includes Docker Engine, Docker Compose, Docker Content Trust, Kubernetes, and Credential Helper.
- **For Linux**: You can install Docker Engine directly on your Linux distribution. Docker provides a script that 
makes installation straightforward.

### 2. Basic Docker Commands

After installing Docker, familiarize yourself with some basic commands:

- **`docker --version`**: Check the version of Docker installed on your system.
- **`docker run -it ubuntu /bin/bash`**: This command downloads the Ubuntu image (if it's not already present) and 
runs a container from it, opening a bash shell inside the container. The `-it` flags allow you to interact with the
container.
- **`docker ps`**: Lists all running containers.
- **`docker ps -a`**: Lists all containers, including those that are stopped.
- **`docker stop <container_id>`**: Stops a running container.
- **`docker rm <container_id>`**: Removes a stopped container.
- **`docker images`**: Lists all available Docker images on your system.
- **`docker rmi <image_id>`**: Removes a Docker image.

### 3. Docker Images and Containers

- **Images**: Docker images are templates for creating Docker containers. You can pull images from Docker Hub or 
create your own.
- **Containers**: Containers are runtime instances of images. You can run multiple containers from a single image.

### 4. Dockerfile

A Dockerfile is a text document that contains all the commands a user could call on the command line to assemble an
image. Here's a simple example of a Dockerfile that builds an image for a Python application:

```Dockerfile
# Use an official Python runtime as a parent image
FROM python:3.9-slim

# Set the working directory in the container
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
RUN pip install --trusted-host pypi.python.org -r requirements.txt

# Make port 80 available to the world outside this container
EXPOSE 80

# Define environment variable
ENV NAME World

# Run app.py when the container launches
CMD ["python", "app.py"]
```

### 5. Building and Running Your Docker Image

- **Build your image**: `docker build -t my-python-app .`
- **Run your container**: `docker run -p 4000:80 my-python-app`

This guide covers the basics of using Docker. Docker is a vast and powerful tool, so there's much more to learn, 
including Docker Compose for managing multiple containers, Docker Swarm for orchestration, and Kubernetes for more 
complex deployments.

In [39]:
with open('synthetic_data.jsonl', 'w') as f:
    for item in instruction_response_pair_list:
        f.write(json.dumps(item))
        f.write('\n')

In [40]:
messages = [
    {
        "role": "user",
        "content": "What is the chemical formula for water?"
    },
    {
        "role": "assistant",
        "content": "The chemical formula for water is H₂O."
    },
]

response = client.chat.completions.create(
    model="nvidia/nemotron-4-340b-reward",
    messages=messages,
)
print(response.choices[0].logprobs.content)

[
    ChatCompletionTokenLogprob(token='helpfulness', bytes=None, logprob=4.0625, top_logprobs=[]),
    ChatCompletionTokenLogprob(token='correctness', bytes=None, logprob=3.921875, top_logprobs=[]),
    ChatCompletionTokenLogprob(token='coherence', bytes=None, logprob=4.0625, top_logprobs=[]),
    ChatCompletionTokenLogprob(token='complexity', bytes=None, logprob=0.55078125, top_logprobs=[]),
    ChatCompletionTokenLogprob(token='verbosity', bytes=None, logprob=0.443359375, top_logprobs=[])
]

In [41]:
def get_scores_from_response(openai_response_template):
    logprobs = openai_response_template.choices[0].logprobs.content
    score_dict = {}
    for score in logprobs:
        score_dict[score.token] = score.logprob
    return score_dict

print(get_scores_from_response(response))

{
    'helpfulness': 4.0625,
    'correctness': 3.921875,
    'coherence': 4.0625,
    'complexity': 0.55078125,
    'verbosity': 0.443359375
}

In [42]:
def get_response_and_scores(client, model, question, response_content):
    messages = [
        {
            "role": "user",
            "content": question
        },
        {
            "role": "assistant",
            "content": response_content
        },
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    scores = get_scores_from_response(response)
    return scores

In [43]:
def parse_synthetic_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            json_entry = json.loads(line)
            data.append({
                    "instruction": json_entry["instruction"],
                    "response": json_entry["response"]
                })
    return data

synthetic_data = parse_synthetic_data("synthetic_data.jsonl")
len(synthetic_data)

482

In [44]:
def process_instruction_response_pairs(client, model, synthetic_data):
    scores = []
    for instruction_response_pair in synthetic_data:
        instruction = instruction_response_pair["instruction"]
        response = instruction_response_pair['response']
        scores.append(get_response_and_scores(client, model, instruction, response))
    return scores


scores = process_instruction_response_pairs(client, "nvidia/nemotron-4-340b-reward", synthetic_data)

helpfulness_THRESHOLD = 3
verbosity_THRESHOLD = 2.5
synthetic_data = [data for i, data in enumerate(synthetic_data) 
                  if not (scores[i]["helpfulness"] < helpfulness_THRESHOLD or 
                          scores[i]["verbosity"] > verbosity_THRESHOLD)]
len(synthetic_data)

339

In [45]:
with open('synthetic_data_filtered.jsonl', 'w') as f:
    for item in synthetic_data:
        f.write(json.dumps(item))
        f.write('\n')

In [4]:
with open(f'synthetic_data_filtered.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]
dataset = Dataset.from_list(data)
dataset_dict = DatasetDict({"train": dataset})
dataset_dict.push_to_hub("Shafagh99/docker-prompt")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Shafagh99/docker-prompt/commit/0802d58693ec6d0d8a4bbdeb57410f8bbffe0d7a', commit_message='Upload dataset', commit_description='', oid='0802d58693ec6d0d8a4bbdeb57410f8bbffe0d7a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Shafagh99/docker-prompt', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Shafagh99/docker-prompt'), pr_revision=None, pr_num=None)